In [60]:
import tifffile
import numpy as np
from tqdm import tqdm
from pathlib import Path
import os
import pandas as pd
import sys
sys.path.append(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\10_code\UTvsXCT-preprocessing')
from dbtools import dbtools as db
from dbtools import load
from preprocess_tools import io, aligner, reslicer

# Database conection

In [61]:
try:
    conn = db.connect()
    print("Connected to the database")
except Exception as error:
    print(error)

Connected to the database


## Measurements file paths

List of paths to the XCT measurements you aim to align

In [62]:
paths = [Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\4\volume_eq_rotated'),
         Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\5\volume_eq_rotated'),
         Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\7\volume_eq'),
         Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\8\volume_eq'),
         Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\10\volume_eq'),
         Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\11\volume_eq'),
         Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\12\volume_eq')]

#check that the paths exist
for path in paths:
    if not path.exists():
        print(f"Path {path} does not exist")

# Measurement type id

Get the measurement id of each measurement

In [63]:
measurements_table = db.get_data('measurements')

measurmenttype_ids = []

for path in paths:
    row = measurements_table[measurements_table['file_path_measurement'] == str(path)]
    if not row.empty:
        measurmenttype_ids.append(int(row['measurementtype_id_measurement'].values[0]))
    else:
        print(f"Measurement not found for path: {path}")

print("Measurement type ids:", measurmenttype_ids)

Measurement type ids: [2, 2, 2, 2, 2, 2, 2]


# Sample names

Get the samples names in the measurements

In [64]:
measurements_samples_table = db.relation_metadata('samples','measurements','sample_measurements')

measurements_samples_table

,id_sample,name_sample,panel_id_sample,height_sample,width_sample,thickness_sample,keyhole_sample,parallel_faces_sample,id_measurement,file_path_measurement,parent_measurement_id_measurement,measurementtype_id_measurement,height_measurement,width_measurement,depth_measurement,dtype_measurement,file_type_measurement,signal_type_measurement,aligned_measurement,zflip_measurement,vflip_measurement,equalized_measurement,axes_measurement,transformation_interpolation_order_measurement
0,1,JI_4,1,150 mm,40 mm,5 mm,True Boolean,parallel nominal,1,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,NaN,3,505 cardinal,203 cardinal,372 cardinal,uint8 nominal,tif nominal,RF nominal,NaN,NaN,NaN,NaN,NaN,NaN
1,1,JI_4,1,150 mm,40 mm,5 mm,True Boolean,parallel nominal,3,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,1.0,3,505 cardinal,120 cardinal,45 cardinal,uint8 nominal,tif nominal,RF nominal,NaN,NaN,NaN,NaN,NaN,NaN
2,1,JI_4,1,150 mm,40 mm,5 mm,True Boolean,parallel nominal,21,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,NaN,2,3737 cardinal,1770 cardinal,1015 cardinal,uint16 nominal,folder nominal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,JI_4,1,150 mm,40 mm,5 mm,True Boolean,parallel nominal,22,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,21.0,2,3224 cardinal,1770 cardinal,1015 cardinal,uint8 nominal,folder nominal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,JI_4,1,150 mm,40 mm,5 mm,True Boolean,parallel nominal,26,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,22.0,2,3224 cardinal,1610 cardinal,240 cardinal,uint8 nominal,folder nominal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,8,JI_12,1,150 mm,40 mm,5 mm,True Boolean,parallel nominal,24,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,23.0,2,3226 cardinal,1734 cardinal,1003 cardinal,uint8 nominal,folder nominal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,8,JI_12,1,150 mm,40 mm,5 mm,True Boolean,parallel nominal,33,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,24.0,2,3226 cardinal,1626 cardinal,250 cardinal,uint8 nominal,folder nominal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,8,JI_12,1,150 mm,40 mm,5 mm,True Boolean,parallel nominal,34,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,33.0,2,219 cardinal,3233 cardinal,1620 cardinal,uint8 nominal,folder nominal,NaN,True boolean,NaN,NaN,NaN,NaN,NaN
68,8,JI_12,1,150 mm,40 mm,5 mm,True Boolean,parallel nominal,43,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,NaN,4,448 cardinal,190 cardinal,492 cardinal,uint8 nominal,tif nominal,RF nominal,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:

sample_names = []

for path in paths:
    sub_table = measurements_samples_table[measurements_samples_table['file_path_measurement'] == str(path)]
    if not sub_table.empty:
        sample_names.append(sub_table['name_sample'].values)
    else:
        print(f"Measurement not found for path: {path}")

print("Measurement sample names:", sample_names)

Measurement sample names: [array(['JI_4'], dtype=object), array(['JI_5'], dtype=object), array(['JI_7'], dtype=object), array(['JI_8'], dtype=object), array(['JI_10'], dtype=object), array(['JI_11'], dtype=object), array(['JI_12'], dtype=object)]


# Alignment and saving

Loop that loads each measurement, aligns the volume and loads it in the database.

In [66]:
for i, path in enumerate(paths):
    print(f"Processing {path} with measurement type id {measurmenttype_ids[i]}")
    
    # Load the data
    volume = io.load_tif(path)
    
    # Align the volume
    volume = aligner.main(volume)

    #reslice to frontal 
    volume = reslicer.rotate_90(volume, clockwise=True)
    volume = reslicer.reslice(volume,'Top')
    
    # Save the aligned volume as .tif if is less than 4GB if its more as a folder
    if volume.nbytes < 4 * 1024**3:  # Check if the size is less than 4GB
        aligned_path = path.parent / f"{path.name}_aligned.tif"
        file_type = 'tif'
    else:
        # Save as a folder
        aligned_path = path.parent / f"{path.name}_aligned"
        if not aligned_path.exists():
            aligned_path.mkdir(parents=True, exist_ok=True)
        file_type = 'tif folder'
    
    print(f"Saving aligned volume to {aligned_path}")

    io.save_tif(aligned_path,volume)

    #data for the database

    height, width, depth = volume.shape

    dtype = str(volume.dtype)

    samples = list(sample_names[i])

    equalized = True

    transformation = 'Aligned with lucias method. V01'

    print(f"Inserting measurement for path {aligned_path} with type {file_type}, height {height}, width {width}, depth {depth}, dtype {dtype}, samples {samples}, equalized {equalized}, transformation {transformation}")

    # Save the aligned volume in the database
    inserted_id = load.load_xct_measurement(conn,str(aligned_path),measurmenttype_ids[i],height,width,depth,dtype,file_type,samples,True, equalized,['z','y','x'],str(path),transformation)

    print(f"Inserted measurement with id {inserted_id} for path {aligned_path}")

Processing \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\4\volume_eq_rotated with measurement type id 2


Rotating around y-axis: 100%|██████████| 3228/3228 [00:10<00:00, 295.60it/s]


computing otsu
Bounding box coordinates: zmin=1, zmax=3226, ymin=1, ymax=1609, xmin=55, xmax=257
Saving aligned volume to \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\4\volume_eq_rotated_aligned.tif
Inserting measurement for path \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\4\volume_eq_rotated_aligned.tif with type tif, height 203, width 3226, depth 1609, dtype uint8, samples ['JI_4'], equalized True, transformation Aligned with lucias method. V01
XCT measurement from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\4\volume_eq_rotated_aligned.tif' loaded with ID: 93
Inserted measurement with id 93 for path \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\4\volume_eq_rotated_aligned.tif
Processing \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\5\volume_eq_rotated with measurement type id 2


Rotating around x-axis:   0%|          | 0/1629 [00:00<?, ?it/s]c:\Users\alberto.vicente\.conda\envs\prepUTvsXCT\Lib\site-packages\joblib\externals\loky\process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Rotating around y-axis: 100%|██████████| 3228/3228 [00:10<00:00, 302.40it/s]


computing otsu
Bounding box coordinates: zmin=1, zmax=3226, ymin=1, ymax=1628, xmin=55, xmax=246
Saving aligned volume to \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\5\volume_eq_rotated_aligned.tif
Inserting measurement for path \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\5\volume_eq_rotated_aligned.tif with type tif, height 192, width 3226, depth 1628, dtype uint8, samples ['JI_5'], equalized True, transformation Aligned with lucias method. V01
XCT measurement from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\5\volume_eq_rotated_aligned.tif' loaded with ID: 94
Inserted measurement with id 94 for path \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\5\volume_eq_rotated_aligned.tif
Processing \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\7\volume_eq with measurement type id 2


...

computing otsu
Bounding box coordinates: zmin=1, zmax=3227, ymin=4, ymax=1582, xmin=80, xmax=284
Saving aligned volume to \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\10\volume_eq_aligned.tif
Inserting measurement for path \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\10\volume_eq_aligned.tif with type tif, height 205, width 3227, depth 1579, dtype uint8, samples ['JI_10'], equalized True, transformation Aligned with lucias method. V01
XCT measurement from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\10\volume_eq_aligned.tif' loaded with ID: 97
Inserted measurement with id 97 for path \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\10\volume_eq_aligned.tif
Processing \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\11\volume_eq with measurement type id 2


Rotating around y-axis: 100%|██████████| 3228/3228 [00:10<00:00, 321.00it/s]


computing otsu
Bounding box coordinates: zmin=0, zmax=3227, ymin=13, ymax=1611, xmin=51, xmax=253
Saving aligned volume to \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\11\volume_eq_aligned.tif
Inserting measurement for path \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\11\volume_eq_aligned.tif with type tif, height 203, width 3228, depth 1599, dtype uint8, samples ['JI_11'], equalized True, transformation Aligned with lucias method. V01
XCT measurement from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\11\volume_eq_aligned.tif' loaded with ID: 98
Inserted measurement with id 98 for path \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\11\volume_eq_aligned.tif
Processing \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\12\volume_eq with measurement type id 2


Rotating around y-axis: 100%|██████████| 3229/3229 [00:10<00:00, 315.07it/s]


computing otsu
Bounding box coordinates: zmin=1, zmax=3227, ymin=17, ymax=1609, xmin=59, xmax=258
Saving aligned volume to \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\12\volume_eq_aligned.tif
Inserting measurement for path \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\12\volume_eq_aligned.tif with type tif, height 200, width 3227, depth 1593, dtype uint8, samples ['JI_12'], equalized True, transformation Aligned with lucias method. V01
XCT measurement from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\12\volume_eq_aligned.tif' loaded with ID: 99
Inserted measurement with id 99 for path \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\12\volume_eq_aligned.tif
